In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top-instagram-influencers-data-cleaned/top_insta_influencers_data.csv


In [2]:
df = pd.read_csv('/kaggle/input/top-instagram-influencers-data-cleaned/top_insta_influencers_data.csv')

In [3]:
df.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3.3k,475.8m,8.7m,1.39%,6.5m,29.0b,Spain
1,2,kyliejenner,91,6.9k,366.2m,8.3m,1.62%,5.9m,57.4b,United States
2,3,leomessi,90,0.89k,357.3m,6.8m,1.24%,4.4m,6.0b,NaN
3,4,selenagomez,93,1.8k,342.7m,6.2m,0.97%,3.3m,11.5b,United States
4,5,therock,91,6.8k,334.1m,1.9m,0.20%,665.3k,12.5b,United States


In [4]:
df.isnull().sum()

rank                  0
channel_info          0
influence_score       0
posts                 0
followers             0
avg_likes             0
60_day_eng_rate       0
new_post_avg_like     0
total_likes           0
country              62
dtype: int64

### Data cleaning 

In [5]:
# Step 1: Handling Numeric Columns
# Many of our columns have values stored as strings with 'k', 'm', and 'b' (thousands, millions, billions). We need to convert them into proper numeric values.

def convert_numeric(value):
    if isinstance(value, str): # ensure its a string before processing
        if 'k' in value:
            return float(value.replace('k', '')) * 1e3
        elif 'm' in value:
            return float(value.replace('m', '')) * 1e6
        elif 'b' in value:
            return float(value.replace('b','')) * 1e9
        else:
            return float(value)
    return value

In [6]:
# Apply conversion to numeric columns
numeric_columns = ["posts", "followers", "avg_likes", "new_post_avg_like", "total_likes"]
for col in numeric_columns:
    df[col] = df[col].apply(convert_numeric).astype(float) # convert to float explicitly 
df.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3300.0,475800000.0,8700000.0,1.39%,6500000.0,2.900000e+10,Spain
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,1.62%,5900000.0,5.740000e+10,United States
2,3,leomessi,90,890.0,357300000.0,6800000.0,1.24%,4400000.0,6.000000e+09,NaN
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.97%,3300000.0,1.150000e+10,United States
4,5,therock,91,6800.0,334100000.0,1900000.0,0.20%,665300.0,1.250000e+10,United States


In [7]:
# convert engagement rate to decimal format 
df['60_day_eng_rate'] = df['60_day_eng_rate'].str.replace('%', '').astype(float)/100
df.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States
2,3,leomessi,90,890.0,357300000.0,6800000.0,0.0124,4400000.0,6.000000e+09,NaN
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States
4,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States


In [8]:
# renaming columns 
df.rename(columns = {'channel_info': 'influencer_name', '60_day_eng_rate': 'engagement_rate'}, inplace = True)
df.head()

,rank,influencer_name,influence_score,posts,followers,avg_likes,engagement_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States
2,3,leomessi,90,890.0,357300000.0,6800000.0,0.0124,4400000.0,6.000000e+09,NaN
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States
4,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States


In [9]:
# find the most frequent country name 
most_frequent_country = df['country'].mode()[0] # get the most common country 
df['country'].fillna(most_frequent_country, inplace = True)
df.head()

<ipython-input-9-247c9a8dbc87>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].fillna(most_frequent_country, inplace = True)


,rank,influencer_name,influence_score,posts,followers,avg_likes,engagement_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States
2,3,leomessi,90,890.0,357300000.0,6800000.0,0.0124,4400000.0,6.000000e+09,United States
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States
4,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States


In [10]:
df.isnull().sum()

rank                 0
influencer_name      0
influence_score      0
posts                0
followers            0
avg_likes            0
engagement_rate      1
new_post_avg_like    0
total_likes          0
country              0
dtype: int64

In [11]:
# Retrieve the row where engagement_rate is null
missing_engagement_rate_row = df[df["engagement_rate"].isnull()]
missing_engagement_rate_row

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,rank,influencer_name,influence_score,posts,followers,avg_likes,engagement_rate,new_post_avg_like,total_likes,country
167,168,rkive,83,110.0,37000000.0,10900000.0,NaN,0.0,1.200000e+09,United States


In [12]:
# Calculate the mean engagement rate (excluding NaN values)
mean_engagement_rate = df["engagement_rate"].mean()

# Fill missing values with the mean engagement rate
# df["engagement_rate"].fillna(mean_engagement_rate, inplace=True)
df = df.fillna({"engagement_rate": mean_engagement_rate})
df.head()

,rank,influencer_name,influence_score,posts,followers,avg_likes,engagement_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States
2,3,leomessi,90,890.0,357300000.0,6800000.0,0.0124,4400000.0,6.000000e+09,United States
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States
4,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States


# SQL 

In [13]:
import sqlite3
# Create an SQLite database in memory
conn = sqlite3.connect(":memory:")  #  creates an in-memory SQLite database
cursor = conn.cursor()  # Explicitly create a cursor 

# Load cleaned dataset into SQLite
df.to_sql("instagram_influencers", conn, index=False, if_exists="replace")

200

In [14]:
# Count the Number of Influencers per Country
query = """
SELECT country, COUNT(*) AS influencer_count
FROM instagram_influencers
GROUP BY country
ORDER BY influencer_count DESC
Limit 10;
"""
influencer_country = pd.read_sql(query, conn)
display(influencer_country)

,country,influencer_count
0,United States,128
1,Brazil,13
2,India,12
3,Indonesia,7
4,France,6
5,Spain,5
6,United Kingdom,4
7,Colombia,3
8,Canada,3
9,Turkey,2


In [15]:
# Find the Top 10 Influencers by Engagement Rate
query = """
SELECT influencer_name, followers, avg_likes, engagement_rate
FROM instagram_influencers
ORDER BY engagement_rate DESC
LIMIT 10;
"""
influencer_engagement = pd.read_sql(query, conn)
display(influencer_engagement)

,influencer_name,followers,avg_likes,engagement_rate
0,j.m,41900000.0,14200000.0,0.2641
1,thv,49300000.0,15400000.0,0.2580
2,badbunnypr,42100000.0,3700000.0,0.1309
3,tomholland2013,67700000.0,5400000.0,0.1083
4,karolg,55600000.0,3100000.0,0.1025
5,roses_are_rosie,61800000.0,4600000.0,0.0972
6,sooyaaa__,62900000.0,4500000.0,0.0943
7,lalalalisa_m,80900000.0,5800000.0,0.0900
8,zayn,46500000.0,4700000.0,0.0881
9,milliebobbybrown,57600000.0,4000000.0,0.0863


In [16]:
# Rank Influencers by Total Likes
query = """
SELECT influencer_name, total_likes, rank
FROM instagram_influencers
ORDER BY total_likes DESC
LIMIT 10;
"""
rank_influencer_like = pd.read_sql(query, conn)
display(rank_influencer_like)

,influencer_name,total_likes,rank
0,kyliejenner,5.740000e+10,2
1,cristiano,2.900000e+10,1
2,zendaya,2.060000e+10,23
3,kimkardashian,1.990000e+10,6
4,arianagrande,1.840000e+10,7
5,badgalriri,1.790000e+10,26
6,neymarjr,1.410000e+10,20
7,justinbieber,1.390000e+10,10
8,nickiminaj,1.350000e+10,17
9,therock,1.250000e+10,5


In [17]:
# Which Country Has the Most Total Likes?
# Instead of just counting influencers per country, we sum total likes to see which country dominates influencer marketing.
query = """
SELECT country, SUM(total_likes) AS total_likes_sum
FROM instagram_influencers
GROUP BY country
ORDER BY total_likes_sum DESC
limit 10;
"""
country_like = pd.read_sql(query, conn)
display(country_like)

,country,total_likes_sum
0,United States,4.909850e+11
1,Brazil,4.096930e+10
2,Spain,3.868120e+10
3,India,3.291900e+10
4,Canada,2.550000e+10
5,Netherlands,1.790000e+10
6,Colombia,1.510000e+10
7,Indonesia,1.416570e+10
8,United Kingdom,1.350000e+10
9,France,1.047130e+10


In [18]:
# Which Influencers Have the Best Engagement Relative to Followers?
# Engagement Score = avg_likes / followers
# This finds underrated influencers with high engagement but fewer followers.
query = """
SELECT influencer_name, followers, avg_likes, 
       ROUND((avg_likes / followers) * 100, 2) AS engagement_score
FROM instagram_influencers
ORDER BY engagement_score DESC
LIMIT 10;
"""
influencer_engagement = pd.read_sql_query(query, conn)
display(influencer_engagement)

,influencer_name,followers,avg_likes,engagement_score
0,j.m,41900000.0,14200000.0,33.89
1,thv,49300000.0,15400000.0,31.24
2,rkive,37000000.0,10900000.0,29.46
3,jenniferaniston,40700000.0,4600000.0,11.30
4,mahi7781,39100000.0,4100000.0,10.49
5,zayn,46500000.0,4700000.0,10.11
6,harrystyles,46900000.0,4700000.0,10.02
7,adele,50700000.0,4700000.0,9.27
8,blakelively,34600000.0,3100000.0,8.96
9,badbunnypr,42100000.0,3700000.0,8.79


In [19]:
# Which Influencers Have the Fastest Growth Potential?
# New Post Average Likes vs. Total Likes
# This shows influencers who get more likes on recent posts, meaning they’re growing.
query = """
SELECT influencer_name, total_likes, new_post_avg_like, 
       ROUND((new_post_avg_like / total_likes) * 100, 2) AS growth_potential
FROM instagram_influencers
ORDER BY growth_potential DESC
LIMIT 10;
"""
rising_star = pd.read_sql_query(query, conn)
display(rising_star)

,influencer_name,total_likes,new_post_avg_like,growth_potential
0,badbunnypr,6.750000e+07,5400000.0,8.00
1,j.m,3.681000e+08,11000000.0,2.99
2,daddyyankee,1.830000e+07,455300.0,2.49
3,nattinatasha,1.870000e+07,285900.0,1.53
4,thv,9.874000e+08,12600000.0,1.28
5,zayn,7.735000e+08,4000000.0,0.52
6,milliebobbybrown,1.100000e+09,5000000.0,0.45
7,ddlovato,9.130000e+07,363400.0,0.40
8,deepikapadukone,4.190000e+08,1500000.0,0.36
9,adidasoriginals,2.120000e+07,66800.0,0.32


In [20]:
# Find Anomalies in Engagement Rate
# If an influencer’s engagement rate is much lower than expected, it might indicate fake followers or declining popularity.
query = """
SELECT influencer_name, followers, avg_likes, engagement_rate
FROM instagram_influencers
WHERE engagement_rate < (SELECT AVG(engagement_rate) FROM instagram_influencers) / 2
ORDER BY engagement_rate ASC
limit 10;
"""
anomaly = pd.read_sql_query(query, conn)
display(anomaly)

,influencer_name,followers,avg_likes,engagement_rate
0,emmawatson,67300000.0,1900000.0,0.0001
1,marinaruybarbosa,40700000.0,586400.0,0.0001
2,badgalriri,135300000.0,3700000.0,0.0002
3,willsmith,63900000.0,1400000.0,0.0002
4,buzzfeedtasty,43200000.0,65100.0,0.0002
5,wesleysafadao,37400000.0,256100.0,0.0002
6,blakelively,34600000.0,3100000.0,0.0002
7,5-Minute Crafts GIRLY,46500000.0,157000.0,0.0003
8,victoriassecret,73200000.0,147000.0,0.0004
9,hudabeauty,50800000.0,186400.0,0.0004


In [21]:
# Most Active Influencers (Based on Posts)
# Who posts the most content?
query = """
SELECT influencer_name, posts, total_likes
FROM instagram_influencers
ORDER BY posts DESC
LIMIT 10;
"""
most_active_influencer = pd.read_sql_query(query, conn)
display(most_active_influencer)

,influencer_name,posts,total_likes
0,raffinagita1717,17500.0,6.800000e+09
1,natgeotravel,16900.0,3.200000e+09
2,ruben_onsu,13400.0,1.900000e+09
3,nba,12900.0,4.800000e+09
4,worldstar,11600.0,1.800000e+09
5,433,10300.0,8.900000e+09
6,natgeo,10000.0,3.000000e+09
7,theellenshow,10000.0,4.200000e+09
8,fcbarcelona,10000.0,1.160000e+10
9,snoopdogg,10000.0,2.000000e+09


#### Feature Engineering in SQL

In [22]:
# Compute "Likes per Post" for Each Influencer
query = """
SELECT influencer_name, total_likes, posts, 
       ROUND(total_likes / posts, 2) AS likes_per_post
FROM instagram_influencers
ORDER BY likes_per_post DESC
LIMIT 10;
"""
likes_per_post = pd.read_sql_query(query, conn)
display(likes_per_post)

,influencer_name,total_likes,posts,likes_per_post
0,j.m,3.681000e+08,20.0,18405000.00
1,thv,9.874000e+08,60.0,16456666.67
2,rkive,1.200000e+09,110.0,10909090.91
3,cristiano,2.900000e+10,3300.0,8787878.79
4,billieeilish,5.900000e+09,690.0,8550724.64
5,kyliejenner,5.740000e+10,6900.0,8318840.58
6,badbunnypr,6.750000e+07,10.0,6750000.00
7,leomessi,6.000000e+09,890.0,6741573.03
8,selenagomez,1.150000e+10,1800.0,6388888.89
9,zendaya,2.060000e+10,3500.0,5885714.29


In [23]:
# Predicting Engagement Using SQL
# Create a dummy prediction of future engagement based on past trends.
query = """
SELECT influencer_name, followers, avg_likes, engagement_rate,
       ROUND((engagement_rate * followers * 1.1), 0) AS predicted_engagement
FROM instagram_influencers
ORDER BY predicted_engagement DESC
LIMIT 10;
"""
predicting_engagement = pd.read_sql_query(query, conn)
display(predicting_engagement)

,influencer_name,followers,avg_likes,engagement_rate,predicted_engagement
0,thv,49300000.0,15400000.0,0.2580,13991340.0
1,j.m,41900000.0,14200000.0,0.2641,12172369.0
2,tomholland2013,67700000.0,5400000.0,0.1083,8065101.0
3,lalalalisa_m,80900000.0,5800000.0,0.0900,8009100.0
4,cristiano,475800000.0,8700000.0,0.0139,7274982.0
5,roses_are_rosie,61800000.0,4600000.0,0.0972,6607656.0
6,kyliejenner,366200000.0,8300000.0,0.0162,6525684.0
7,sooyaaa__,62900000.0,4500000.0,0.0943,6524617.0
8,jennierubyjane,68900000.0,5100000.0,0.0836,6336044.0
9,karolg,55600000.0,3100000.0,0.1025,6268900.0


In [24]:
# Categorizing Influencers into Tiers (Micro, Macro, Mega)
# Influencers can be categorized based on followers:
# Rising Star: <50M followers
# Popular: 50M - 200M followers
# Global Icon: >200M followers

query = """
SELECT influencer_name, followers,
    CASE 
        WHEN followers < 50000000 THEN 'Rising Star'
        WHEN followers BETWEEN 50000000 AND 200000000 THEN 'Popular'
        ELSE 'Global Icon'
    END AS influencer_tier
FROM instagram_influencers
ORDER BY followers DESC;

"""
influencer_strata = pd.read_sql_query(query, conn)
display(influencer_strata)

,influencer_name,followers,influencer_tier
0,cristiano,475800000.0,Global Icon
1,kyliejenner,366200000.0,Global Icon
2,leomessi,357300000.0,Global Icon
3,selenagomez,342700000.0,Global Icon
4,therock,334100000.0,Global Icon
...,...,...,...
195,iambeckyg,33200000.0,Rising Star
196,nancyajram,33200000.0,Rising Star
197,luansantana,33200000.0,Rising Star
198,nickjonas,33000000.0,Rising Star


In [25]:
# Top 5 influencers from each category (Global Icon, Popular, Rising Star):

query = """
WITH RankedInfluencers AS (
    SELECT influencer_name, followers,
        CASE 
            WHEN followers < 50000000 THEN 'Rising Star'
            WHEN followers BETWEEN 50000000 AND 200000000 THEN 'Popular'
            ELSE 'Global Icon'
        END AS influencer_tier,
        ROW_NUMBER() OVER (PARTITION BY 
            CASE 
                WHEN followers < 50000000 THEN 'Rising Star'
                WHEN followers BETWEEN 50000000 AND 200000000 THEN 'Popular'
                ELSE 'Global Icon'
            END ORDER BY followers DESC) AS rank
    FROM instagram_influencers
)
SELECT influencer_name, followers, influencer_tier
FROM RankedInfluencers
WHERE rank <= 5;
"""
rank_influencer = pd.read_sql_query(query, conn)
display(rank_influencer)

,influencer_name,followers,influencer_tier
0,cristiano,475800000.0,Global Icon
1,kyliejenner,366200000.0,Global Icon
2,leomessi,357300000.0,Global Icon
3,selenagomez,342700000.0,Global Icon
4,therock,334100000.0,Global Icon
5,kourtneykardash,195200000.0,Popular
6,mileycyrus,181500000.0,Popular
7,neymarjr,177100000.0,Popular
8,katyperry,170300000.0,Popular
9,kevinhart4real,152000000.0,Popular


In [26]:
# What Is the Engagement Trend of Top Influencers?
# Calculates likes per post and engagement rate over time to detect trends.

query = """
SELECT influencer_name, 
       ROUND(total_likes / posts, 2) AS likes_per_post, 
       engagement_rate,
       RANK() OVER (ORDER BY engagement_rate DESC) AS engagement_rank
FROM instagram_influencers
ORDER BY likes_per_post DESC
LIMIT 10;
"""
engagement_trend = pd.read_sql_query(query, conn)
display(engagement_trend)

,influencer_name,likes_per_post,engagement_rate,engagement_rank
0,j.m,18405000.00,0.26410,1
1,thv,16456666.67,0.25800,2
2,rkive,10909090.91,0.01902,53
3,cristiano,8787878.79,0.01390,72
4,billieeilish,8550724.64,0.05020,18
5,kyliejenner,8318840.58,0.01620,60
6,badbunnypr,6750000.00,0.13090,3
7,leomessi,6741573.03,0.01240,75
8,selenagomez,6388888.89,0.00970,96
9,zendaya,5885714.29,0.03170,28


In [27]:
# How Do Recent Posts Compare to Overall Engagement?
# If new posts are getting fewer likes than overall avg_likes, it might indicate declining engagement.
# If recent posts are performing worse than overall likes, it could suggest audience disengagement.

query = """
SELECT influencer_name, avg_likes, new_post_avg_like,
       ROUND(((new_post_avg_like - avg_likes) / avg_likes) * 100, 2) AS engagement_change_percent
FROM instagram_influencers
ORDER BY engagement_change_percent ASC
LIMIT 10;
"""
recent_posts_overall_engagement = pd.read_sql_query(query, conn)
display(recent_posts_overall_engagement)

,influencer_name,avg_likes,new_post_avg_like,engagement_change_percent
0,badgalriri,3700000.0,0.0,-100.00
1,emmawatson,1900000.0,0.0,-100.00
2,willsmith,1400000.0,0.0,-100.00
3,marinaruybarbosa,586400.0,0.0,-100.00
4,mahi7781,4100000.0,0.0,-100.00
5,wesleysafadao,256100.0,0.0,-100.00
6,rkive,10900000.0,0.0,-100.00
7,blakelively,3100000.0,0.0,-100.00
8,5-Minute Crafts GIRLY,157000.0,12400.0,-92.10
9,hudabeauty,186400.0,16800.0,-90.99


In [28]:
# Predicting Future Engagement Based on Growth Rate
# Forecasts engagement rate growth over time using past trends.
query = """
SELECT influencer_name, followers, engagement_rate,
       ROUND((engagement_rate * 1.05), 4) AS predicted_engagement_next_period
FROM instagram_influencers
ORDER BY predicted_engagement_next_period DESC
LIMIT 10;
"""
future_engagement = pd.read_sql_query(query, conn)
display(future_engagement)

,influencer_name,followers,engagement_rate,predicted_engagement_next_period
0,j.m,41900000.0,0.2641,0.2773
1,thv,49300000.0,0.2580,0.2709
2,badbunnypr,42100000.0,0.1309,0.1374
3,tomholland2013,67700000.0,0.1083,0.1137
4,karolg,55600000.0,0.1025,0.1076
5,roses_are_rosie,61800000.0,0.0972,0.1021
6,sooyaaa__,62900000.0,0.0943,0.0990
7,lalalalisa_m,80900000.0,0.0900,0.0945
8,zayn,46500000.0,0.0881,0.0925
9,milliebobbybrown,57600000.0,0.0863,0.0906


In [29]:
# Which Influencers Have the Best ROI for Brand Deals?
# Calculates Cost per Engagement (CPE) (lower is better for advertisers).
query = """
SELECT influencer_name, followers, avg_likes, engagement_rate, 
       ROUND((followers / avg_likes), 2) AS cost_per_engagement
FROM instagram_influencers
ORDER BY cost_per_engagement ASC
LIMIT 10;
"""
best_roi = pd.read_sql_query(query, conn)
display(best_roi)

,influencer_name,followers,avg_likes,engagement_rate,cost_per_engagement
0,j.m,41900000.0,14200000.0,0.26410,2.95
1,thv,49300000.0,15400000.0,0.25800,3.20
2,rkive,37000000.0,10900000.0,0.01902,3.39
3,jenniferaniston,40700000.0,4600000.0,0.01880,8.85
4,mahi7781,39100000.0,4100000.0,0.00170,9.54
5,zayn,46500000.0,4700000.0,0.08810,9.89
6,harrystyles,46900000.0,4700000.0,0.06380,9.98
7,adele,50700000.0,4700000.0,0.03820,10.79
8,blakelively,34600000.0,3100000.0,0.00020,11.16
9,badbunnypr,42100000.0,3700000.0,0.13090,11.38


In [30]:
# Which Countries Have the Most Valuable Influencer Market?
# Aggregates influencer engagement rates by country.
query = """
SELECT country, COUNT(influencer_name) AS total_influencers,
       ROUND(AVG(engagement_rate), 4) AS avg_engagement_rate
FROM instagram_influencers
GROUP BY country
ORDER BY avg_engagement_rate DESC;
"""
country_influencer_market = pd.read_sql_query(query, conn)
display(country_influencer_market)

,country,total_influencers,avg_engagement_rate
0,Uruguay,1,0.0540
1,Mexico,2,0.0436
2,CÃ´te d'Ivoire,1,0.0382
3,Australia,1,0.0369
4,India,12,0.0237
5,Canada,3,0.0235
6,United States,128,0.0226
7,Anguilla,1,0.0176
8,Netherlands,2,0.0131
9,United Kingdom,4,0.0104


In [31]:
# Are Any Influencers Losing Followers at an Unusual Rate?
query = """
SELECT influencer_name, followers, avg_likes, engagement_rate
FROM instagram_influencers
WHERE engagement_rate < (SELECT AVG(engagement_rate) FROM instagram_influencers) / 2
ORDER BY engagement_rate ASC;
"""
losing_followers = pd.read_sql_query(query, conn)
display(losing_followers)

,influencer_name,followers,avg_likes,engagement_rate
0,emmawatson,67300000.0,1900000.0,0.0001
1,marinaruybarbosa,40700000.0,586400.0,0.0001
2,badgalriri,135300000.0,3700000.0,0.0002
3,willsmith,63900000.0,1400000.0,0.0002
4,buzzfeedtasty,43200000.0,65100.0,0.0002
...,...,...,...,...
97,nattinatasha,35900000.0,518100.0,0.0080
98,paulpogba,55200000.0,1400000.0,0.0085
99,kimkardashian,329200000.0,3500000.0,0.0088
100,krisjenner,49700000.0,356800.0,0.0091
